In [1]:
import numpy as np
import torch
import cv2
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from pandas.core.common import flatten
from torchvision import datasets, transforms, models
import random

In [ ]:
num_workers = 0
batch_size = 20
valid_size = 0.2
transform_train = transforms.Compose([transforms.Resize((64,64)),
									  transforms.Grayscale(),
									  transforms.ToTensor(),
    								  transforms.Normalize((0.5), (0.5))])

transform_test = transforms.Compose([transforms.Resize((64,64)),
									 transforms.Grayscale(),
									 transforms.ToTensor(),
									 transforms.Normalize((0.5), (0.5))])
train_data = torchvision.datasets.ImageFolder('~/Desktop/Handwritten Arithmetic Calculator/All Data/Small Dataset Thresholded/Train', transform=transform_train)
test_data = torchvision.datasets.ImageFolder('~/Desktop/Handwritten Arithmetic Calculator/All Data/Handwritten Dataset Thresholded/Train', transform=transform_test)
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,num_workers=num_workers,shuffle = True)
classes=['+','-','0','1','2','3','4','5','6','7','8','9','div','times']

In [ ]:
class Network(nn.Module):
	def __init__(self):
		super(Network,self).__init__()
		self.cl1=nn.Conv2d(1,16,3,padding=1)
		self.cl1_bn=nn.BatchNorm2d(16)
		self.cl2=nn.Conv2d(16,32,3,padding=1)
		self.cl2_bn=nn.BatchNorm2d(32)
		self.cl3=nn.Conv2d(32,64,3,padding=1)
		self.cl3_bn=nn.BatchNorm2d(64)
		self.dropout=nn.Dropout(0.2)
		self.fc1=nn.Linear(4096,1024)
		self.fc1_bn=nn.BatchNorm1d(1024)
		self.fc2=nn.Linear(1024,256)
		self.fc2_bn=nn.BatchNorm1d(256)
		self.fc3=nn.Linear(256,14)
		self.pool=nn.MaxPool2d(2,2)

	def forward(self,x):
		x=self.cl1(x)
		x=self.pool(F.leaky_relu(self.cl1_bn(x)))
		x=self.cl2(x)
		x=self.pool(F.leaky_relu(self.cl2_bn(x)))
		x=self.cl3(x)
		x=self.pool(F.leaky_relu(self.cl3_bn(x)))
		x=x.view(-1,4096)
		self.dropout(x)
		x=self.fc1(x)
		x=F.relu(self.fc1_bn(x))
		self.dropout(x)
		x=self.fc2(x)
		x=F.relu(self.fc2_bn(x))
		self.dropout(x)
		x=self.fc3(x)
		return x

model=Network()
model.cuda()

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.0005)

In [ ]:
%matplotlib inline
def imshow(img):
    img = img / 2 + 0.5
    plt.imshow(np.transpose(img, (1, 2, 0)))  
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() 
fig = plt.figure(figsize=(25, 4))

for idx in np.arange(20):
    ax = fig.add_subplot(2, 20//2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

In [ ]:
torch.cuda.empty_cache()

In [ ]:
epochs=50
train_losses=np.array([])
valid_losses=np.array([])
ep=np.array([])
valid_loss_min = np.Inf
for e in range(epochs):
	train_loss=0.0
	valid_loss=0.0
	model.train()
	for data, target in train_loader:
		data, target = data.cuda(), target.cuda()
		# data = torchvision.transforms.functional.invert(data)
		output=model(data)
		optimizer.zero_grad()
		loss=criterion(output,target)
		loss.backward()
		optimizer.step()
		train_loss+=loss.item()*data.size(0)
	model.eval()
	for data, target in valid_loader:
		data, target = data.cuda(), target.cuda()
		output = model(data)
		loss = criterion(output, target)																
		valid_loss += loss.item()*data.size(0)   
	train_loss = train_loss/len(train_loader.sampler)
	valid_loss = valid_loss/len(valid_loader.sampler)
	train_losses=np.append(train_losses,train_loss)
	valid_losses=np.append(valid_losses,valid_loss)
	ep=np.append(ep,e)
	print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
		e, train_loss, valid_loss))   
	if valid_loss <= valid_loss_min:
		print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
		torch.save(model.state_dict(), 'model_ERA_lr0005_2.pt')
		valid_loss_min = valid_loss

In [ ]:
plt.plot(ep,train_losses,'o-')
plt.plot(ep,valid_losses,'o-')
plt.xlabel('Epochs')
plt.ylabel('Losses')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Losses')
plt.show()

In [ ]:
model.load_state_dict(torch.load('model_ERA_lr0005_2.pt'))

In [ ]:
test_loss = 0.0
class_correct = list(0. for i in range(14))
class_total = list(0. for i in range(14))
model.eval()
for data, target in test_loader:
    data, target = data.cuda(), target.cuda()
    # data = torchvision.transforms.functional.invert(data)
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)   
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    
    for i in range(0,target.size()[0]):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))
                
for i in range(14):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))


print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))